In [1]:
#Tensorflow
import tensorflow as tf
from tensorflow.keras import Sequential
from tensorflow import keras
from tensorflow.keras.models import Model
from tensorflow.keras.models import load_model
from tensorflow.keras import layers, activations
from tensorflow.keras.layers import *
from tensorflow.keras.optimizers import *

#Pandas and numpy for data formats
import pandas as pd
import numpy as np


#glob for data import
import glob
import random
#PyTorch
import torch
import torch.nn as nn
#SK Learn
import sklearn as skl
from sklearn.model_selection import train_test_split
from sklearn import preprocessing
from sklearn.preprocessing import StandardScaler
#from sklearn.utils import shuffle
import math
import sys
#MatPlotLib
import matplotlib.pyplot as plt

from models import simpleRegression
from models import feedForward, sequentialModel, sequentialDropout, lstmModel, cnnModel, cnnModelFilters, scoringSystem

#import functions
from dataImport import importTrainingData, importParamsData, importTestData, importTrainParameter, importTargetParameter, importEvalParameter
from preprocessing import preprocessData, createTrainTensors, createParamsTensors, preprocessParams, rescaleData, preprocessParameters


#importTrainData()
train_data = np.load("../train_40000.npy")
params_data = np.load("../train_target_40000.npy")
train_parameter = np.load("../train_parameter_40000.npy")
#
validation_data = np.load("../val_train_40000.npy")
validation_params = np.load("../val_target_40000.npy")
validation_parameter = np.load("../val_parameter_40000.npy")

#Create stats for zScore
mean_value_target = np.mean(params_data)
standard_deviation_target = np.std(params_data)

mean_value = np.mean(train_data)
standard_deviation = np.std(train_data)
measurements_to_remove = 30


median_train = False
replaceValuesBiggerOne_train=False
replaceFirstXValues_train=False
removeFirstXValues_train=True
normalizeData_train=False
zScore_train=True

train_data = preprocessData(train_data,
                          median=median_train,
                          replaceValuesBiggerOne = replaceValuesBiggerOne_train,
                          replaceFirstXValues = replaceFirstXValues_train,
                          removeFirstXValues = removeFirstXValues_train,
                          normalizeData = normalizeData_train,
                          zScore = zScore_train,
                          zScore_mean = mean_value,
                          zScore_sd = standard_deviation,
                          number_to_remove = measurements_to_remove)

train_target = preprocessData(params_data,
                          median=False,
                          replaceValuesBiggerOne=False,
                          replaceFirstXValues=False,
                          removeFirstXValues=False,
                          normalizeData=False,
                          zScore=False,
                          zScore_mean=mean_value_target,
                          zScore_sd=standard_deviation_target,
                          number_to_remove=measurements_to_remove)

validation_data = preprocessData(validation_data,
                          median = median_train,
                          replaceValuesBiggerOne = replaceValuesBiggerOne_train,
                          replaceFirstXValues = replaceFirstXValues_train,
                          removeFirstXValues = removeFirstXValues_train,
                          normalizeData = normalizeData_train,
                          zScore = zScore_train,
                          zScore_mean = mean_value,
                          zScore_sd = standard_deviation,
                          number_to_remove = measurements_to_remove)

validation_target = preprocessData(validation_params,
                          median=False,
                          replaceValuesBiggerOne=False,
                          replaceFirstXValues=False,
                          removeFirstXValues=False,
                          normalizeData=False,
                          zScore=False,
                          zScore_mean=mean_value_target,
                          zScore_sd=standard_deviation_target,
                          number_to_remove=measurements_to_remove)


number_of_measurements = 300
if removeFirstXValues_train == True:
    number_of_measurements = 300 - measurements_to_remove    
    
#reshaping train data
if type(train_data) != np.ndarray:
    train_data = train_data.to_numpy().reshape(-1, 55, number_of_measurements)
else:
    train_data = train_data.reshape(-1, 55, number_of_measurements)

#reshaping target data
if type(params_data) != np.ndarray:
    params_data = params_data.to_numpy()

#reshaping val data
if type(validation_data) != np.ndarray:
    validation_data = validation_data.to_numpy().reshape(-1, 55, number_of_measurements)
else:    
    validation_data = validation_data.reshape(-1, 55, number_of_measurements)

#reshaping val target
if type(validation_params) != np.ndarray:
    validation_params = validation_params.to_numpy()

    
def scaleToMaxToOne(dat, min, max):
    scaled_dat = dat / max
    return scaled_dat

def unscaleMaxToOne(scaled_dat, min, max):
    unscaled_dat = scaled_dat * max
    return unscaled_dat

min_target = np.min(train_target) 
max_target = np.max(train_target)
print(f"Rescaling targets with max = {max_target}")
train_target = scaleToMaxToOne(train_target, min_target, max_target)
val_target = scaleToMaxToOne(validation_target, min_target, max_target)    

prevent_overfitting = keras.callbacks.EarlyStopping(monitor="val_loss", patience = 20, restore_best_weights=True)
reduce_lr = keras.callbacks.ReduceLROnPlateau(monitor='val_loss', patience=10, min_delta=0.0001)

INFO:tensorflow:Enabling eager execution
INFO:tensorflow:Enabling v2 tensorshape
INFO:tensorflow:Enabling resource variables
INFO:tensorflow:Enabling tensor equality
INFO:tensorflow:Enabling control flow v2
Rescaling targets with max = 0.6713607926990712


In [2]:
train_parameter = train_parameter.reshape(-1, 6)
validation_parameter = validation_parameter.reshape(-1, 6)

In [3]:
validation_parameter.shape

(10000, 6)

In [4]:
def create_concated_model(av = 'selu'):
    first_input = Input(shape=(55, number_of_measurements))

    second_input = Input(shape=(6,))

    first_dense = Dense(1024, activation=av)(first_input)
    second_dense = Dense(1024, activation=av)(first_dense)
    third_dense = Dense(1024, activation=av)(second_dense)
    fourth_dense = Dense(1024, activation=av)(third_dense)
    flatten_layer = tf.keras.layers.Flatten()(fourth_dense)


    parameter_first_dense = Dense(10, activation=av)(second_input)
    parameter_second_dense = Dense(10, activation=av)(parameter_first_dense)
    parameter_third_dense = Dense(10, activation=av)(parameter_second_dense)
    parameter_fourth_dense = Dense(10, activation=av)(parameter_third_dense)
    
    concatLayer = tf.keras.layers.concatenate([parameter_fourth_dense, flatten_layer]) 


    output = Dense(55)(concatLayer)


    model = Model(inputs=[first_input, second_input], outputs=output)
    optim = keras.optimizers.Adam(learning_rate=0.0001)
    
    model.compile(optimizer=optim,
                  loss='mae')
        
    return model

model = create_concated_model()

history = model.fit([train_data, train_parameter],
                    train_target,
                    batch_size = 32,
                    epochs = 1000,
                    validation_split = 0.2,
                    shuffle = True,
                    callbacks = [prevent_overfitting, reduce_lr])



Epoch 1/1000
1000/1000 [==============================] - 12s 7ms/step - loss: 0.2550 - val_loss: 0.0786
Epoch 2/1000
1000/1000 [==============================] - 7s 7ms/step - loss: 0.0619 - val_loss: 0.0459
Epoch 3/1000
1000/1000 [==============================] - 7s 7ms/step - loss: 0.0394 - val_loss: 0.0363
Epoch 4/1000
1000/1000 [==============================] - 7s 7ms/step - loss: 0.0305 - val_loss: 0.0289
Epoch 5/1000
1000/1000 [==============================] - 7s 7ms/step - loss: 0.0253 - val_loss: 0.0242
Epoch 6/1000
1000/1000 [==============================] - 7s 7ms/step - loss: 0.0213 - val_loss: 0.0214
Epoch 7/1000
1000/1000 [==============================] - 7s 7ms/step - loss: 0.0188 - val_loss: 0.0196
Epoch 8/1000
1000/1000 [==============================] - 7s 7ms/step - loss: 0.0168 - val_loss: 0.0170
Epoch 9/1000
1000/1000 [==============================] - 7s 7ms/step - loss: 0.0154 - val_loss: 0.0162
Epoch 10/1000
1000/1000 [==============================] - 7s 7

1000/1000 [==============================] - 7s 7ms/step - loss: 0.0017 - val_loss: 0.0053
Epoch 80/1000
1000/1000 [==============================] - 7s 7ms/step - loss: 0.0017 - val_loss: 0.0056
Epoch 81/1000
1000/1000 [==============================] - 7s 7ms/step - loss: 0.0017 - val_loss: 0.0055
Epoch 82/1000
1000/1000 [==============================] - 7s 7ms/step - loss: 0.0017 - val_loss: 0.0054
Epoch 83/1000
1000/1000 [==============================] - 7s 7ms/step - loss: 0.0016 - val_loss: 0.0055
Epoch 84/1000
1000/1000 [==============================] - 7s 7ms/step - loss: 0.0016 - val_loss: 0.0055
Epoch 85/1000
1000/1000 [==============================] - 7s 7ms/step - loss: 0.0016 - val_loss: 0.0056
Epoch 86/1000
1000/1000 [==============================] - 7s 7ms/step - loss: 0.0015 - val_loss: 0.0055
Epoch 87/1000
1000/1000 [==============================] - 7s 7ms/step - loss: 0.0015 - val_loss: 0.0054
Epoch 88/1000
1000/1000 [==============================] - 7s 7ms/ste

In [5]:
pred = model.predict([validation_data, validation_parameter])
#pred = pred * standard_deviation_target + mean_value_target 


pred_final = unscaleMaxToOne(pred, min_target, max_target)
score = scoringSystem(validation_params, pred_final)
print(score)

tf.Tensor(550000.0, shape=(), dtype=float64)
tf.Tensor(9753.99714500089, shape=(), dtype=float64)


In [6]:
validation_data.shape

(10000, 55, 270)

In [7]:
validation_parameter.shape

(10000, 6)

In [8]:
train_data.shape


(40000, 55, 270)

In [9]:
train_parameter.shape

(40000, 6)